In [1]:
import pandas as pd
import pyvi
import spacy
from spacy.training import Example
from spacy.util import minibatch, compounding
import random
import json

In [2]:
def train_ner_model(train_df, n_iter=30):
    """
    Huấn luyện NER model từ một DataFrame (đọc từ file .jsonl).
    """
    
    # --- 1. Chuyển đổi DataFrame sang định dạng list (text, annot) ---
    # Đây là bước quan trọng để hàm tương thích với dữ liệu từ DataFrame
    TRAIN_DATA = []
    for _, row in train_df.iterrows():
        TRAIN_DATA.append(
            (row['text'], {"entities": row['entities']})
        )
    
    # --- 2. Tạo model và pipeline ---
    # Dòng này yêu cầu thư viện 'pyvi' phải được cài đặt
    nlp = spacy.blank("vi")
    print("Đã tạo mô hình 'vi' trống.")
    
    if "ner" not in nlp.pipe_names:
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    
    # --- 3. TỰ ĐỘNG THÊM TẤT CẢ LABEL (Thay đổi quan trọng) ---
    # Tự động tìm tất cả các nhãn duy nhất từ dữ liệu
    all_labels = set()
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            all_labels.add(ent[2]) # ent[2] là nhãn (vd: 'SYMPTOM', 'DISEASE')
            
    for label in all_labels:
        ner.add_label(label)
        
    print(f"Đã thêm các nhãn vào pipeline: {all_labels}")

    # --- 4. Bắt đầu training (Giữ nguyên logic của bạn) ---
    print("🚀 Bắt đầu huấn luyện NER model...")
    
    optimizer = nlp.begin_training()
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    
    with nlp.disable_pipes(*other_pipes):
        for iteration in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            
            # Sử dụng TRAIN_DATA (list đã chuyển đổi) ở đây
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            
            for batch in batches:
                examples = []
                for text, annotations in batch:
                    try:
                        doc = nlp.make_doc(text)
                        example = Example.from_dict(doc, annotations)
                        examples.append(example)
                    except Exception as e:
                        # Bỏ qua nếu có lỗi chú thích (ví dụ: entity nằm ngoài câu)
                        print(f"Lỗi khi tạo Example: {e} - Dữ liệu: {text[:50]}...")
                
                # Cập nhật model
                if examples: # Chỉ update nếu có examples hợp lệ
                    nlp.update(examples, drop=0.35, sgd=optimizer, losses=losses)
            
            # Chỉ in loss nếu có 'ner' trong losses
            if (iteration + 1) % 5 == 0 and 'ner' in losses:
                print(f"Iteration {iteration + 1}/{n_iter} - Loss: {losses['ner']:.4f}")
    
    print("✓ Hoàn thành training!")
    return nlp

In [3]:
df = pd.read_json("../data/processed/train_data.spacy.jsonl", lines=True)
print(df.head())

                                               text  \
0                         Tôi bị đau đầu và sốt nhẹ   
1               Mấy hôm nay tôi ho khan và đau họng   
2           Tôi bị sổ mũi và nghẹt mũi mấy ngày nay   
3                  Cơ thể tôi rất mệt mỏi và uể oải   
4  Tôi có triệu chứng ớn lạnh và đau nhức toàn thân   

                                 entities  
0   [[7, 14, SYMPTOM], [18, 25, SYMPTOM]]  
1  [[16, 23, SYMPTOM], [27, 35, SYMPTOM]]  
2   [[7, 13, SYMPTOM], [17, 26, SYMPTOM]]  
3  [[15, 22, SYMPTOM], [26, 32, SYMPTOM]]  
4  [[19, 26, SYMPTOM], [30, 48, SYMPTOM]]  


In [4]:
# Training
model = train_ner_model(df, n_iter=30)

# Lưu model
model.to_disk("../models/spacy_ner_model")
print("✓ Đã lưu model vào thư mục 'ner_model'")

Đã tạo mô hình 'vi' trống.
Đã thêm các nhãn vào pipeline: {'DISEASE', 'SYMPTOM'}
🚀 Bắt đầu huấn luyện NER model...
Iteration 5/30 - Loss: 57.3066
Iteration 10/30 - Loss: 40.6455
Iteration 15/30 - Loss: 14.1750
Iteration 20/30 - Loss: 18.7759
Iteration 25/30 - Loss: 8.9134
Iteration 30/30 - Loss: 4.1295
✓ Hoàn thành training!
✓ Đã lưu model vào thư mục 'ner_model'


In [5]:
# Test model
print("\n📋 Test model với câu mẫu:")
test_texts = [
    "Tôi bị sốt và ho nhiều",
    "Em bị đau bụng dữ dội",
    "Bệnh nhân có triệu chứng chóng mặt và buồn nôn"
]

for text in test_texts:
    doc = model(text)
    print(f"\n📝 Input: {text}")
    print(f"🔍 Triệu chứng tìm được:")
    if doc.ents:
        for ent in doc.ents:
            print(f"   - {ent.text} [{ent.label_}]")
    else:
        print("   (Không tìm thấy)")


📋 Test model với câu mẫu:

📝 Input: Tôi bị sốt và ho nhiều
🔍 Triệu chứng tìm được:
   - sốt [SYMPTOM]
   - ho nhiều [SYMPTOM]

📝 Input: Em bị đau bụng dữ dội
🔍 Triệu chứng tìm được:
   - đau bụng dữ dội [SYMPTOM]

📝 Input: Bệnh nhân có triệu chứng chóng mặt và buồn nôn
🔍 Triệu chứng tìm được:
   - buồn nôn [SYMPTOM]
